In [20]:
class SudokuPuzzle:
    def __init__(self, filepath):
        self.grid = self.load_puzzle(filepath)
    
    """Load a Sudoku puzzle from a file and replace '?' with 0."""
    def load_puzzle(self, filepath):
        with open(filepath, 'r', encoding='utf-8-sig') as file:
            lines = [line.strip().replace('?', '0') for line in file]
        grid = []
        for line in lines:
            row = [int(num) for num in line.split(',')]
            grid.append(row)
        return grid

    """Display the Sudoku grid"""
    def display(self):
        for row in self.grid:
            print(" ".join(str(num) if num != 0 else '?' for num in row))

    """Output the Sudoku grid to a file, replacing 0 with '?'"""
    def output_to_file(self, filepath):
        with open(filepath, 'w') as file:
            for row in self.grid:
                line = ",".join(str(num) if num != 0 else '?' for num in row)
                file.write(line + "\n")

    """Check if all rows are valid (no duplicates except 0)"""
    def is_valid_rows(self):
        for row in self.grid:
            nums = [num for num in row if num != 0]
            if len(nums) != len(set(nums)):
                return False
        return True
    
    """Check if all columns are valid (no duplicates except 0)"""
    def is_valid_columns(self):
        for col in range(9):
            nums = [self.grid[row][col] for row in range(9) if self.grid[row][col] != 0]
            if len(nums) != len(set(nums)):
                return False
        return True
    
    """Check if all 3x3 blocks are valid (no duplicates except 0)"""
    def is_valid_blocks(self):
        for block_start_row in [0, 3, 6]:
            for block_start_col in [0, 3, 6]:
                if not self._is_valid_blocks_checker(block_start_row, block_start_col):
                    return False
        return True
    
    """Check if the entire Sudoku grid is valid"""
    def is_valid(self):
        return self.is_valid_rows() and self.is_valid_columns() and self.is_valid_blocks()
    
    """Helper function to check a single 3x3 block"""
    def _is_valid_blocks_checker(self, block_start_row, block_start_col):
        nums = []
        for i in range(3):
            for j in range(3):
                num = self.grid[block_start_row + i][block_start_col + j]
                if num != 0:
                    nums.append(num)
        return len(nums) == len(set(nums))
    
    def set_digit(self, row, col, digit):
        self.grid[row][col] = digit

    
if __name__ == "__main__":
    # Testing the SudokuPuzzle class
    puzzle = SudokuPuzzle('./Puzzles/Easy-P1.txt')
    puzzle.display()
    print("Valid Rows:", puzzle.is_valid_rows())
    print("Valid Columns:", puzzle.is_valid_columns())
    print("Valid Blocks:", puzzle.is_valid_blocks())
    print("Overall Valid:", puzzle.is_valid())

    # Failure case row column
    puzzle.set_digit(0, 0, 6)
    puzzle.display()
    print("Valid Rows:", puzzle.is_valid_rows())
    print("Valid Columns:", puzzle.is_valid_columns())
    print("Valid Blocks:", puzzle.is_valid_blocks())
    print("Overall Valid:", puzzle.is_valid())

    # Failure case blocks
    puzzle.set_digit(0, 1, 7)
    puzzle.display()
    print("Valid Blocks:", puzzle.is_valid_blocks())    
    print("Overall Valid:", puzzle.is_valid())

    # Output to file
    puzzle.output_to_file('./Puzzles/Output-Test.txt')




? ? 8 ? 5 6 ? ? ?
7 ? 4 ? ? ? 6 1 9
? ? ? ? ? ? 8 5 ?
6 ? 7 ? 2 9 5 ? ?
? ? 9 ? 6 ? 1 ? ?
? ? 2 3 1 ? 9 ? 4
? 3 5 ? ? ? ? ? ?
4 2 1 ? ? ? 3 ? 6
? ? ? 8 3 ? 4 ? ?
Valid Rows: True
Valid Columns: True
Valid Blocks: True
Overall Valid: True
6 ? 8 ? 5 6 ? ? ?
7 ? 4 ? ? ? 6 1 9
? ? ? ? ? ? 8 5 ?
6 ? 7 ? 2 9 5 ? ?
? ? 9 ? 6 ? 1 ? ?
? ? 2 3 1 ? 9 ? 4
? 3 5 ? ? ? ? ? ?
4 2 1 ? ? ? 3 ? 6
? ? ? 8 3 ? 4 ? ?
Valid Rows: False
Valid Columns: False
Valid Blocks: True
Overall Valid: False
6 7 8 ? 5 6 ? ? ?
7 ? 4 ? ? ? 6 1 9
? ? ? ? ? ? 8 5 ?
6 ? 7 ? 2 9 5 ? ?
? ? 9 ? 6 ? 1 ? ?
? ? 2 3 1 ? 9 ? 4
? 3 5 ? ? ? ? ? ?
4 2 1 ? ? ? 3 ? 6
? ? ? 8 3 ? 4 ? ?
Valid Blocks: False
Overall Valid: False
